In [0]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!nvidia-smi

Wed Jun  3 09:02:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip uninstall tensorflow
!pip install tensorflow-gpu==1.14.0

# 数据处理

In [0]:
!pip install hanziconv
!pip install pyhanlp
from pyhanlp import *

下载 https://file.hankcs.com/bin/hanlp-1.7.7-release.zip 到 /usr/local/lib/python3.6/dist-packages/pyhanlp/static/hanlp-1.7.7-release.zip
100.00%, 1 MB, 698 KB/s, 还有 0 分  0 秒   
下载 https://file.hankcs.com/hanlp/data-for-1.7.5.zip 到 /usr/local/lib/python3.6/dist-packages/pyhanlp/static/data-for-1.7.7.zip
94.84%, 604 MB, 6579 KB/s, 还有 0 分  5 秒   

In [0]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/fine_gained/mycode
! pwd
! ls

/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
data		 dataset.py   elmo_utils.py  __pycache__
data_not_aug	 elmo.py      output	     run_log.ipynb
data_process.py  elmo_run.py  preprocess.sh  weight_drop_lstm.py


In [0]:
!bash preprocess.sh

In [0]:
# # 未增强的数据
# !python data_process.py \
#     --data_file='../../data/trainingset/sentiment_analysis_trainingset.csv' \
#     --output_file=data_not_aug/train.json \
#     --vocab_file=data_not_aug/vocab.txt \
# 	--is_trian_file=True \
#     --vocab_size=50000

# processed -- 10000 --
# processed -- 20000 --
# processed -- 30000 --
# processed -- 40000 --
# processed -- 50000 --
# processed -- 60000 --
# processed -- 70000 --
# processed -- 80000 --
# processed -- 90000 --
# processed -- 100000 --
# Start to create vocab ...
# Created vocab file data_not_aug/vocab.txt with vocab size 50000


In [0]:
# !python data_process.py \
#     --data_file='../fsauor2018/wordvec/sgns.sogou.word' \
#     --output_file=data_not_aug/embedding.txt \
#     --vocab_file=data_not_aug/vocab.txt \
#     --embedding=True

# !python data_process.py \
#     --data_file='../../data/validationset/sentiment_analysis_validationset.csv' \
#     --output_file=data_not_aug/validation.json

# !python data_process.py \
#     --data_file='../../data/testa/sentiment_analysis_testa.csv' \
#     --output_file=data_not_aug/testa.json

# processed -- 10000 --
# processed -- 10000 --


In [0]:
# import json

# max_len = 0
# count = 0
# with open('data/clip_trian.json', 'w',encoding='utf-8') as f:
#     for line in open('data/train.json'):
#         item = json.loads(line.strip())
#         content = item['content']
#         tokens = content.strip().split()
#         length = len(tokens)
#         if length > max_len:
#             max_len = length
#         if length > 1000:
#             count += 1
#             tokens = tokens[: 500] +  tokens[-500: ]
#         item['content'] = ' '.join(tokens)
#         abc = json.dumps(item, ensure_ascii=False) + '\n'
#         f.write(abc)
        
# print(max_len)
# print(count)

1493
297


In [0]:
# max_len = 0
# count = 0
# for line in open('data/clip_trian.json'):
#         item = json.loads(line.strip())
#         content = item['content']
#         tokens = content.strip().split()
#         length = len(tokens)
#         if length > max_len:
#             max_len = length
#         if length > 1000:
#             count += 1
# print(max_len)
# print(count)

1000
0


# 模型训练

In [29]:
% cd /content/gdrive/My\ Drive/Colab\ Notebooks/nlp_task/fine_gained/mycode
! pwd
! ls

/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
/content/gdrive/My Drive/Colab Notebooks/nlp_task/fine_gained/mycode
data		 dataset.py   elmo_utils.py  __pycache__
data_not_aug	 elmo.py      output	     run_log.ipynb
data_process.py  elmo_run.py  preprocess.sh  weight_drop_lstm.py


In [0]:
# 裁剪前后文字的数据
!python elmo_run.py \
--mode=train \
--data_files=data/clip_trian.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=32 \
--max_len=1000 \
--rnn_cell_name=gru \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=2000 \
--optimizer='adam' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0.1 \
--checkpoint_dir=output/elmo_ema_smoothing_aug

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
# 未裁剪，只截断
!python elmo_run.py \
--mode=train \
--data_files=data/train.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name=lstm \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0.1 \
--checkpoint_dir=output/elmo_1200_aug

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [30]:
# fix embedding + 原始数据
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=64 \
--max_len=1200 \
--rnn_cell_name=lstm \
--num_classes_each_label=4 \
--num_labels=20 \
--fix_embedding=True \
--steps_per_eval=1000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0.1 \
--checkpoint_dir=output/elmo_1200_not_aug_fix \
--checkpoint_load_step=10000

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
# gru效果变差
!python elmo_run.py \
--mode=train \
--data_files=data/train_not_aug.json \
--eval_files=data/validation.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--embed_file=data/embedding.txt \
--num_layers=3 \
--batch_size=32 \
--max_len=1200 \
--rnn_cell_name=gru \
--num_classes_each_label=4 \
--num_labels=20 \
--steps_per_eval=2000 \
--optimizer='rms' \
--learning_rate=0.001 \
--focal_loss_gamma=0 \
--label_smoothing=0.1 \
--checkpoint_dir=output/elmo_ema_smoothing_aug

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
!python elmo_run.py \
--mode=inference \
--data_files=data/testa.json \
--label_file=data/labels.txt \
--vocab_file=data/vocab.txt \
--out_file=output/out.testa.json \
--prob=False \
--batch_size=100 \
--num_classes_each_label 4 \
--num_labels=20 \
--checkpoint_dir=output/elmo_ema_smoothing_aug